In [3]:
import csv
import cv2
import numpy as np

lines = []
with open("driving_log.csv") as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

images = []
measurements = []
counter = 0
for line in lines:
    sourcepath = line[0]
    filename = sourcepath.split('\\')[-1]
    currentpath = "IMG/" + filename
    image = cv2.imread(currentpath)
    images.append(image)
    measurements.append(line[3])
    counter = counter + 1
    
print(images[0].shape)
x_train = np.array(images)
y_train = np.array(measurements)

from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D

model = Sequential()
model.add(Lambda(lambda x: (x/255.0), input_shape = (160, 320, 3)))
model.add(Convolution2D(6,5,5,activation="relu"))
model.add(MaxPooling2D())
model.add(Convolution2D(6,5,5,activation="relu"))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(120))
model.add(Dense(84))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')
model.fit(x_train, y_train, validation_split=0.2, shuffle = True, epochs = 10)
model.save('model.h5')

scores = model.evaluate(x_train, y_train)
print(scores)

(160, 320, 3)


/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:34: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(6, (5, 5), activation="relu")`
/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:36: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(6, (5, 5), activation="relu")`


Train on 1548 samples, validate on 388 samples
Epoch 1/10
1548/1548 [==============================] - 9s - loss: 76.1450 - val_loss: 0.0200
Epoch 2/10
1548/1548 [==============================] - 6s - loss: 0.0245 - val_loss: 0.0195
Epoch 3/10
1548/1548 [==============================] - 6s - loss: 0.0240 - val_loss: 0.0189
Epoch 4/10
1548/1548 [==============================] - 6s - loss: 0.0235 - val_loss: 0.0184
Epoch 5/10
1548/1548 [==============================] - 6s - loss: 0.0230 - val_loss: 0.0180
Epoch 6/10
1548/1548 [==============================] - 6s - loss: 0.0227 - val_loss: 0.0178
Epoch 7/10
1548/1548 [==============================] - 6s - loss: 0.0223 - val_loss: 0.0176
Epoch 8/10
1548/1548 [==============================] - 6s - loss: 0.0219 - val_loss: 0.0174
Epoch 9/10
1548/1548 [==============================] - 6s - loss: 0.0216 - val_loss: 0.0172
Epoch 10/10
1936/1936 [==============================] - 2s     
0.0200674313569
